# Weather related data 2013-01-01 to 2017-11-29

In [1]:
from __future__ import print_function, division
%matplotlib inline
import os
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import geopandas as gpd
import requests
import json
from io import StringIO
import io
import shapely

# Step1 clean

## 1. weather data - Local Climatological Data (LCD)

In [2]:
df = pd.read_csv('data/weather130101-171130.csv')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (10,11,14,15,20,23,25,40,44,45,67,68,84,85,86,87) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
print ("Shape of data\n{}".format(df.shape))

Shape of data
(55996, 90)


In [4]:
df.drop(['STATION', 'STATION_NAME', 'ELEVATION', 'LATITUDE', 
              'LONGITUDE', 'REPORTTPYE', 'HOURLYSKYCONDITIONS', 
              'HOURLYVISIBILITY', 'HOURLYPRSENTWEATHERTYPE', 
              'HOURLYDRYBULBTEMPF', 'HOURLYDRYBULBTEMPC', 
              'HOURLYWETBULBTEMPF', 'HOURLYWETBULBTEMPC', 
              'HOURLYDewPointTempF', 'HOURLYDewPointTempC', 
              'HOURLYRelativeHumidity', 'HOURLYWindDirection',
              'HOURLYPrecip', 'MonthlyMaximumTemp',
       'MonthlyMinimumTemp', 'MonthlyMeanTemp', 'MonthlyAverageRH',
       'MonthlyDewpointTemp', 'MonthlyWetBulbTemp',
       'MonthlyAvgHeatingDegreeDays', 'MonthlyAvgCoolingDegreeDays',
       'MonthlyStationPressure', 'MonthlySeaLevelPressure',
       'MonthlyAverageWindSpeed', 'MonthlyTotalSnowfall',
       'MonthlyDeptFromNormalMaximumTemp', 'MonthlyDeptFromNormalMinimumTemp',
       'MonthlyDeptFromNormalAverageTemp', 'MonthlyDeptFromNormalPrecip',
       'MonthlyTotalLiquidPrecip', 'MonthlyGreatestPrecip',
       'MonthlyGreatestPrecipDate', 'MonthlyGreatestSnowfall',
       'MonthlyGreatestSnowfallDate', 'MonthlyGreatestSnowDepth',
       'MonthlyGreatestSnowDepthDate', 'MonthlyDaysWithGT90Temp',
       'MonthlyDaysWithLT32Temp', 'MonthlyDaysWithGT32Temp',
       'MonthlyDaysWithLT0Temp', 'MonthlyDaysWithGT001Precip',
       'MonthlyDaysWithGT010Precip', 'MonthlyDaysWithGT1Snow',
       'MonthlyMaxSeaLevelPressureValue', 'MonthlyMaxSeaLevelPressureDate',
       'MonthlyMaxSeaLevelPressureTime', 'MonthlyMinSeaLevelPressureValue',
       'MonthlyMinSeaLevelPressureDate', 'MonthlyMinSeaLevelPressureTime',
       'MonthlyTotalHeatingDegreeDays', 'MonthlyTotalCoolingDegreeDays',
       'MonthlyDeptFromNormalHeatingDD', 'MonthlyDeptFromNormalCoolingDD',
       'MonthlyTotalSeasonToDateHeatingDD',
       'MonthlyTotalSeasonToDateCoolingDD','DAILYAverageStationPressure',
       'DAILYAverageSeaLevelPressure', 'DAILYAverageWindSpeed',
       'DAILYPeakWindSpeed', 'PeakWindDirection', 'DAILYSustainedWindSpeed',
       'DAILYSustainedWindDirection','DAILYMaximumDryBulbTemp',
       'DAILYMinimumDryBulbTemp', 'DAILYAverageDryBulbTemp',
       'DAILYDeptFromNormalAverageTemp', 'DAILYAverageRelativeHumidity',
       'DAILYAverageDewPointTemp', 'DAILYAverageWetBulbTemp',
       'DAILYHeatingDegreeDays', 'DAILYCoolingDegreeDays'], axis=1, inplace=True)

In [5]:
df.columns

Index(['DATE', 'HOURLYWindSpeed', 'HOURLYWindGustSpeed',
       'HOURLYStationPressure', 'HOURLYPressureTendency',
       'HOURLYPressureChange', 'HOURLYSeaLevelPressure',
       'HOURLYAltimeterSetting', 'DAILYSunrise', 'DAILYSunset', 'DAILYWeather',
       'DAILYPrecip', 'DAILYSnowfall', 'DAILYSnowDepth'],
      dtype='object')

In [6]:
df.count()

DATE                      55996
HOURLYWindSpeed           50749
HOURLYWindGustSpeed        9362
HOURLYStationPressure     53604
HOURLYPressureTendency    14083
HOURLYPressureChange       2423
HOURLYSeaLevelPressure    42318
HOURLYAltimeterSetting    53660
DAILYSunrise              55996
DAILYSunset               55996
DAILYWeather                776
DAILYPrecip                2388
DAILYSnowfall              1794
DAILYSnowDepth             1794
dtype: int64

In [7]:
df.head(1)

,DATE,HOURLYWindSpeed,HOURLYWindGustSpeed,HOURLYStationPressure,HOURLYPressureTendency,HOURLYPressureChange,HOURLYSeaLevelPressure,HOURLYAltimeterSetting,DAILYSunrise,DAILYSunset,DAILYWeather,DAILYPrecip,DAILYSnowfall,DAILYSnowDepth
0,2013-01-01 00:51,10.0,22.0,29.72,8.0,NaN,29.87,29.89,720,1639,NaN,NaN,NaN,NaN


df.HOURLYWindSpeed.unique()
df.HOURLYStationPressure.unique()
df.HOURLYWindGustSpeed.unique()
df.HOURLYPressureTendency.unique()
df.HOURLYPressureChange.unique()
df.HOURLYSeaLevelPressure.unique()
df.HOURLYAltimeterSetting.unique()
df.DAILYWeather.unique()
#T indicates trace amount.
df.DAILYPrecip.unique()
#The depth of frozen precipitation like snow is measured in tenths (0.1) of an inch and 
#trace amounts are defined as less than half that amount (0.05 inch).
#T indicates trace amount.
df.DAILYSnowfall.unique()
#T indicates trace amount.
df.DAILYSnowDepth.unique()
df.DAILYSunrise.unique()
df.DAILYSunset.unique()

In [8]:
df1 = df[['DATE', 'DAILYSunrise', 'DAILYSunset']]
df1.head(1)

,DATE,DAILYSunrise,DAILYSunset
0,2013-01-01 00:51,720,1639


In [9]:
df1['DATE'] = pd.to_datetime(df1['DATE'], format='%Y/%m/%d')
df1['DATE'] = df1['DATE'].dt.date
df1['DATE'] = pd.to_datetime(df1['DATE'], format='%Y/%m/%d')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

In [10]:
df1.dtypes

DATE            datetime64[ns]
DAILYSunrise             int64
DAILYSunset              int64
dtype: object

In [11]:
df1.drop_duplicates(subset='DATE', inplace=True)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [12]:
df1.reset_index(drop=True, inplace=True)
df1.head()

,DATE,DAILYSunrise,DAILYSunset
0,2013-01-01,720,1639
1,2013-01-02,720,1640
2,2013-01-03,720,1641
3,2013-01-04,720,1642
4,2013-01-05,720,1643


In [13]:
df1.shape

(1795, 3)

In [14]:
df1.dtypes

DATE            datetime64[ns]
DAILYSunrise             int64
DAILYSunset              int64
dtype: object

In [15]:
df1.DATE.unique()

array(['2013-01-01T00:00:00.000000000', '2013-01-02T00:00:00.000000000',
       '2013-01-03T00:00:00.000000000', ...,
       '2017-11-28T00:00:00.000000000', '2017-11-29T00:00:00.000000000',
       '2017-11-30T00:00:00.000000000'], dtype='datetime64[ns]')

## 2. weather data -- daily summary

In [16]:
df2 = pd.read_csv('data/weather_daily_summary_2013_now.csv')

In [17]:
print ("Shape of data\n{}".format(df2.shape))

Shape of data
(1794, 27)


In [18]:
df2.count()

STATION      1794
NAME         1794
LATITUDE     1794
LONGITUDE    1794
ELEVATION    1794
DATE         1794
AWND         1781
PGTM            1
PRCP         1794
SNOW         1793
SNWD         1793
TAVG            0
TMAX         1794
TMIN         1794
WSF2         1784
WSF5         1768
WT01          508
WT02           32
WT04            5
WT06            7
WT08          347
WT13           27
WT14            1
WT16           23
WT18           21
WT19            9
WT22            2
dtype: int64

In [19]:
df2.head(1)

,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION,DATE,AWND,PGTM,PRCP,SNOW,...,WT02,WT04,WT06,WT08,WT13,WT14,WT16,WT18,WT19,WT22
0,USW00094728,"NY CITY CENTRAL PARK, NY US",40.77898,-73.96925,42.7,2013-01-01,6.93,NaN,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
df2.dtypes

STATION       object
NAME          object
LATITUDE     float64
LONGITUDE    float64
ELEVATION    float64
DATE          object
AWND         float64
PGTM         float64
PRCP         float64
SNOW         float64
SNWD         float64
TAVG         float64
TMAX           int64
TMIN           int64
WSF2         float64
WSF5         float64
WT01         float64
WT02         float64
WT04         float64
WT06         float64
WT08         float64
WT13         float64
WT14         float64
WT16         float64
WT18         float64
WT19         float64
WT22         float64
dtype: object

In [21]:
df2.drop(['STATION', 'NAME', 'LATITUDE', 'LONGITUDE', 'ELEVATION'], axis=1, inplace=True)
df2.columns = ['DATE', 'wind_speed_average',
       'peak_gust_time', 'precipitation', 'snowfall', 'snow_depth', 'temperature_average', 'temperature_max', 'temperature_min', 'fastest_2minute_windspeed', 'fastest_5minute_windspeed',
       'fog', 'heavy_fog', 'ice_pellets', 'glaze_rime', 'smoke_haze', 'mist', 'drizzle', 'rain', 'WT18',
       'precipitation_unknownsource', 'ice_fog']

In [22]:
df2.drop(['peak_gust_time', 'temperature_average', 'fastest_5minute_windspeed', 'WT18', 'precipitation_unknownsource', 'rain'], axis=1, inplace=True)

In [23]:
df2.count()

DATE                         1794
wind_speed_average           1781
precipitation                1794
snowfall                     1793
snow_depth                   1793
temperature_max              1794
temperature_min              1794
fastest_2minute_windspeed    1784
fog                           508
heavy_fog                      32
ice_pellets                     5
glaze_rime                      7
smoke_haze                    347
mist                           27
drizzle                         1
ice_fog                         2
dtype: int64

In [24]:
df2['DATE'] = pd.to_datetime(df2['DATE'], format='%Y/%m/%d')

In [25]:
df2.dtypes

DATE                         datetime64[ns]
wind_speed_average                  float64
precipitation                       float64
snowfall                            float64
snow_depth                          float64
temperature_max                       int64
temperature_min                       int64
fastest_2minute_windspeed           float64
fog                                 float64
heavy_fog                           float64
ice_pellets                         float64
glaze_rime                          float64
smoke_haze                          float64
mist                                float64
drizzle                             float64
ice_fog                             float64
dtype: object

In [26]:
df2.DATE.unique()

array(['2013-01-01T00:00:00.000000000', '2013-01-02T00:00:00.000000000',
       '2013-01-03T00:00:00.000000000', ...,
       '2017-11-27T00:00:00.000000000', '2017-11-28T00:00:00.000000000',
       '2017-11-29T00:00:00.000000000'], dtype='datetime64[ns]')

## 3. Storm data

In [27]:
dfbad = pd.read_csv('data/storm_data_search_results.csv')
dfbad.head()

,EVENT_ID,CZ_NAME_STR,BEGIN_LOCATION,BEGIN_DATE,BEGIN_TIME,EVENT_TYPE,MAGNITUDE,TOR_F_SCALE,DEATHS_DIRECT,INJURIES_DIRECT,...,BEGIN_RANGE,BEGIN_AZIMUTH,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,ABSOLUTE_ROWNUMBER
0,421883,NEW YORK (MANHATTAN) (ZONE),,01/31/2013,0,High Wind,50.00,,0,0,...,,,,,,,,,,1
1,436900,NEW YORK (MANHATTAN) (ZONE),,02/08/2013,1400,Winter Storm,,,0,0,...,,,,,,,,,,2
2,452589,NEW YORK CO.,NEW YORK,05/11/2013,1508,Thunderstorm Wind,61.00,,0,0,...,1,SW,1,SW,NEW YORK,40.716,-74.0165,40.716,-74.0165,3
3,452588,NEW YORK CO.,NEW YORK,05/11/2013,1508,Thunderstorm Wind,65.00,,0,0,...,2,SSW,2,SSW,NEW YORK,40.7108,-74.0161,40.7108,-74.0161,4
4,459595,NEW YORK CO.,CENTRAL PARK,06/02/2013,2259,Flash Flood,,,0,0,...,2,NNE,2,NNE,CENTRAL PARK,40.8032,-73.9314,40.801,-73.9321,5


In [28]:
dfbad['EVENT_TYPE'].unique()

array(['High Wind', 'Winter Storm', 'Thunderstorm Wind', 'Flash Flood',
       'Flood', 'Excessive Heat', 'Strong Wind', 'Dense Fog',
       'Winter Weather', 'Heavy Snow', 'Blizzard'], dtype=object)

In [29]:
dfbad.shape

(36, 35)

In [30]:
dfbad.columns

Index(['EVENT_ID', 'CZ_NAME_STR', 'BEGIN_LOCATION', 'BEGIN_DATE', 'BEGIN_TIME',
       'EVENT_TYPE', 'MAGNITUDE', 'TOR_F_SCALE', 'DEATHS_DIRECT',
       'INJURIES_DIRECT', 'DAMAGE_PROPERTY_NUM', 'DAMAGE_CROPS_NUM',
       'STATE_ABBR', 'CZ_TIMEZONE', 'MAGNITUDE_TYPE', 'EPISODE_ID', 'CZ_TYPE',
       'CZ_FIPS', 'WFO', 'INJURIES_INDIRECT', 'DEATHS_INDIRECT', 'SOURCE',
       'FLOOD_CAUSE', 'TOR_LENGTH', 'TOR_WIDTH', 'BEGIN_RANGE',
       'BEGIN_AZIMUTH', 'END_RANGE', 'END_AZIMUTH', 'END_LOCATION',
       'BEGIN_LAT', 'BEGIN_LON', 'END_LAT', 'END_LON', 'ABSOLUTE_ROWNUMBER'],
      dtype='object')

In [31]:
dfbad.MAGNITUDE.unique()

array(['50.00', ' ', '61.00', '65.00', '39.00', '52.00', '30.00', '35.00',
       '48.00'], dtype=object)

In [32]:
dfbad.MAGNITUDE.describe()

count     36
unique     9
top         
freq      24
Name: MAGNITUDE, dtype: object

In [33]:
df3 = dfbad[['BEGIN_DATE', 'EVENT_TYPE']]
df3.head(1)

,BEGIN_DATE,EVENT_TYPE
0,01/31/2013,High Wind


In [34]:
df3.BEGIN_DATE = pd.to_datetime(df3.BEGIN_DATE, format='%m/%d/%Y').dt.strftime('%Y-%m-%d')
df3.BEGIN_DATE = pd.to_datetime(df3.BEGIN_DATE)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [35]:
df3.dtypes

BEGIN_DATE    datetime64[ns]
EVENT_TYPE            object
dtype: object

In [36]:
df3.BEGIN_DATE.unique()

array(['2013-01-31T00:00:00.000000000', '2013-02-08T00:00:00.000000000',
       '2013-05-11T00:00:00.000000000', '2013-06-02T00:00:00.000000000',
       '2013-06-07T00:00:00.000000000', '2013-07-19T00:00:00.000000000',
       '2013-11-01T00:00:00.000000000', '2013-12-05T00:00:00.000000000',
       '2013-12-14T00:00:00.000000000', '2014-01-02T00:00:00.000000000',
       '2014-01-21T00:00:00.000000000', '2014-02-03T00:00:00.000000000',
       '2014-02-05T00:00:00.000000000', '2014-02-13T00:00:00.000000000',
       '2014-07-03T00:00:00.000000000', '2014-07-08T00:00:00.000000000',
       '2015-01-18T00:00:00.000000000', '2015-01-26T00:00:00.000000000',
       '2015-02-01T00:00:00.000000000', '2015-03-05T00:00:00.000000000',
       '2015-04-04T00:00:00.000000000', '2016-01-22T00:00:00.000000000',
       '2016-01-23T00:00:00.000000000', '2016-02-16T00:00:00.000000000',
       '2016-02-24T00:00:00.000000000', '2016-08-12T00:00:00.000000000',
       '2016-08-13T00:00:00.000000000', '2017-01-07

# Step2 Merge

In [37]:
df1.columns

Index(['DATE', 'DAILYSunrise', 'DAILYSunset'], dtype='object')

In [38]:
df2.columns

Index(['DATE', 'wind_speed_average', 'precipitation', 'snowfall', 'snow_depth',
       'temperature_max', 'temperature_min', 'fastest_2minute_windspeed',
       'fog', 'heavy_fog', 'ice_pellets', 'glaze_rime', 'smoke_haze', 'mist',
       'drizzle', 'ice_fog'],
      dtype='object')

In [39]:
df3.columns

Index(['BEGIN_DATE', 'EVENT_TYPE'], dtype='object')

In [40]:
df = df1.merge(df2, left_on='DATE', right_on='DATE', how='left')
df = df.merge(df3, left_on='DATE', right_on='BEGIN_DATE', how='left')
df.drop(['BEGIN_DATE'], axis=1, inplace=True)

In [41]:
df

,DATE,DAILYSunrise,DAILYSunset,wind_speed_average,precipitation,snowfall,snow_depth,temperature_max,temperature_min,fastest_2minute_windspeed,fog,heavy_fog,ice_pellets,glaze_rime,smoke_haze,mist,drizzle,ice_fog,EVENT_TYPE
0,2013-01-01,720,1639,6.93,0.00,0.0,0.0,40.0,26.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013-01-02,720,1640,5.82,0.00,0.0,0.0,33.0,22.0,15.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
2,2013-01-03,720,1641,4.47,0.00,0.0,0.0,32.0,24.0,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2013-01-04,720,1642,8.05,0.00,0.0,0.0,37.0,30.0,17.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2013-01-05,720,1643,6.71,0.00,0.0,0.0,42.0,32.0,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2013-01-06,720,1644,6.71,0.00,0.0,0.0,46.0,34.0,13.0,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN
6,2013-01-07,720,1644,5.37,0.00,0.0,0.0,45.0,37.0,16.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2013-01-08,720,1645,6.49,0.00,0.0,0.0,48.0,35.0,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2013-01-09,720,1646,4.03,0.00,0.0,0.0,49.0,39.0,14.1,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
9,2013-01-10,720,1647,6.71,0.00,0.0,0.0,47.0,40.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
df.shape

(1799, 19)

In [43]:
df.DATE.unique()

array(['2013-01-01T00:00:00.000000000', '2013-01-02T00:00:00.000000000',
       '2013-01-03T00:00:00.000000000', ...,
       '2017-11-28T00:00:00.000000000', '2017-11-29T00:00:00.000000000',
       '2017-11-30T00:00:00.000000000'], dtype='datetime64[ns]')

In [44]:
df[df.duplicated(subset='DATE') == True]

,DATE,DAILYSunrise,DAILYSunset,wind_speed_average,precipitation,snowfall,snow_depth,temperature_max,temperature_min,fastest_2minute_windspeed,fog,heavy_fog,ice_pellets,glaze_rime,smoke_haze,mist,drizzle,ice_fog,EVENT_TYPE
131,2013-05-11,443,1902,3.13,1.09,0.0,0.0,70.0,61.0,10.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Thunderstorm Wind
550,2014-07-03,429,1931,2.91,1.78,0.0,0.0,87.0,69.0,12.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Thunderstorm Wind
1588,2017-05-05,450,1856,6.49,3.02,0.0,0.0,59.0,51.0,16.1,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,Flash Flood
1634,2017-06-19,424,1930,3.80,1.35,0.0,0.0,88.0,70.0,13.0,1.0,1.0,NaN,NaN,1.0,NaN,NaN,NaN,Thunderstorm Wind


In [45]:
df[df.DATE == '2013-05-11']

,DATE,DAILYSunrise,DAILYSunset,wind_speed_average,precipitation,snowfall,snow_depth,temperature_max,temperature_min,fastest_2minute_windspeed,fog,heavy_fog,ice_pellets,glaze_rime,smoke_haze,mist,drizzle,ice_fog,EVENT_TYPE
130,2013-05-11,443,1902,3.13,1.09,0.0,0.0,70.0,61.0,10.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Thunderstorm Wind
131,2013-05-11,443,1902,3.13,1.09,0.0,0.0,70.0,61.0,10.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Thunderstorm Wind


In [46]:
df.drop_duplicates(subset='DATE', inplace=True)

In [47]:
df.shape

(1795, 19)

In [48]:
df.dtypes

DATE                         datetime64[ns]
DAILYSunrise                          int64
DAILYSunset                           int64
wind_speed_average                  float64
precipitation                       float64
snowfall                            float64
snow_depth                          float64
temperature_max                     float64
temperature_min                     float64
fastest_2minute_windspeed           float64
fog                                 float64
heavy_fog                           float64
ice_pellets                         float64
glaze_rime                          float64
smoke_haze                          float64
mist                                float64
drizzle                             float64
ice_fog                             float64
EVENT_TYPE                           object
dtype: object

In [49]:
df.snowfall.unique()

array([  0. ,   1.5,   0.1,   0.3,   0.4,   6.3,   5.1,   4. ,   3. ,
         0.7,   1.4,   5. ,   3.1,   3.3,  11. ,   0.5,   1. ,   0.8,
         8. ,   1.2,   9.5,   1.6,   0.2,   3.6,   5.5,   4.3,   0.9,
         4.8,   1.8,   7.5,   4.5,  27.3,   2.5,   2.8,   0.6,   9.4,
         2.1,   7.6,   nan])

In [50]:
df = df.iloc[:-1, :]
df.shape

(1794, 19)

In [51]:
df.count()

DATE                         1794
DAILYSunrise                 1794
DAILYSunset                  1794
wind_speed_average           1781
precipitation                1794
snowfall                     1793
snow_depth                   1793
temperature_max              1794
temperature_min              1794
fastest_2minute_windspeed    1784
fog                           508
heavy_fog                      32
ice_pellets                     5
glaze_rime                      7
smoke_haze                    347
mist                           27
drizzle                         1
ice_fog                         2
EVENT_TYPE                     32
dtype: int64

In [52]:
df1 = df.iloc[:, :10]
df1.columns

Index(['DATE', 'DAILYSunrise', 'DAILYSunset', 'wind_speed_average',
       'precipitation', 'snowfall', 'snow_depth', 'temperature_max',
       'temperature_min', 'fastest_2minute_windspeed'],
      dtype='object')

In [53]:
nlist = list(df1.columns)[1:]
for i, name in enumerate(nlist):
    print(name)
    df1[name].fillna((df1[name].mean()), inplace=True)

DAILYSunrise
DAILYSunset
wind_speed_average
precipitation
snowfall
snow_depth
temperature_max
temperature_min
fastest_2minute_windspeed


In [54]:
df1.count()

DATE                         1794
DAILYSunrise                 1794
DAILYSunset                  1794
wind_speed_average           1794
precipitation                1794
snowfall                     1794
snow_depth                   1794
temperature_max              1794
temperature_min              1794
fastest_2minute_windspeed    1794
dtype: int64

In [55]:
df1.head()

,DATE,DAILYSunrise,DAILYSunset,wind_speed_average,precipitation,snowfall,snow_depth,temperature_max,temperature_min,fastest_2minute_windspeed
0,2013-01-01,720,1639,6.93,0.0,0.0,0.0,40.0,26.0,15.0
1,2013-01-02,720,1640,5.82,0.0,0.0,0.0,33.0,22.0,15.0
2,2013-01-03,720,1641,4.47,0.0,0.0,0.0,32.0,24.0,13.0
3,2013-01-04,720,1642,8.05,0.0,0.0,0.0,37.0,30.0,17.9
4,2013-01-05,720,1643,6.71,0.0,0.0,0.0,42.0,32.0,17.0


In [56]:
df2 = df.iloc[:, 10:]
df2.columns

Index(['fog', 'heavy_fog', 'ice_pellets', 'glaze_rime', 'smoke_haze', 'mist',
       'drizzle', 'ice_fog', 'EVENT_TYPE'],
      dtype='object')

In [57]:
df2['EVENT_TYPE'] = pd.factorize(df2['EVENT_TYPE'])[0]
df2['EVENT_TYPE'].unique()

array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [58]:
mlist = list(df2.columns)
for i, name in enumerate(mlist):
    print(name)
    df2[name].fillna((0), inplace=True)

fog
heavy_fog
ice_pellets
glaze_rime
smoke_haze
mist
drizzle
ice_fog
EVENT_TYPE


In [59]:
df2.count()

fog            1794
heavy_fog      1794
ice_pellets    1794
glaze_rime     1794
smoke_haze     1794
mist           1794
drizzle        1794
ice_fog        1794
EVENT_TYPE     1794
dtype: int64

In [60]:
df2['EVENT_TYPE'].unique()

array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [61]:
df2.head()

,fog,heavy_fog,ice_pellets,glaze_rime,smoke_haze,mist,drizzle,ice_fog,EVENT_TYPE
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1


In [62]:
df = pd.concat([df1, df2], axis=1)

In [63]:
df.count()

DATE                         1794
DAILYSunrise                 1794
DAILYSunset                  1794
wind_speed_average           1794
precipitation                1794
snowfall                     1794
snow_depth                   1794
temperature_max              1794
temperature_min              1794
fastest_2minute_windspeed    1794
fog                          1794
heavy_fog                    1794
ice_pellets                  1794
glaze_rime                   1794
smoke_haze                   1794
mist                         1794
drizzle                      1794
ice_fog                      1794
EVENT_TYPE                   1794
dtype: int64

In [64]:
df['year'] = df['DATE'].dt.year
df['month'] = df['DATE'].dt.month
df['dayofyear'] = df['DATE'].dt.dayofyear
df.head()

,DATE,DAILYSunrise,DAILYSunset,wind_speed_average,precipitation,snowfall,snow_depth,temperature_max,temperature_min,fastest_2minute_windspeed,...,ice_pellets,glaze_rime,smoke_haze,mist,drizzle,ice_fog,EVENT_TYPE,year,month,dayofyear
0,2013-01-01,720,1639,6.93,0.0,0.0,0.0,40.0,26.0,15.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-1,2013,1,1
1,2013-01-02,720,1640,5.82,0.0,0.0,0.0,33.0,22.0,15.0,...,0.0,0.0,1.0,0.0,0.0,0.0,-1,2013,1,2
2,2013-01-03,720,1641,4.47,0.0,0.0,0.0,32.0,24.0,13.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-1,2013,1,3
3,2013-01-04,720,1642,8.05,0.0,0.0,0.0,37.0,30.0,17.9,...,0.0,0.0,0.0,0.0,0.0,0.0,-1,2013,1,4
4,2013-01-05,720,1643,6.71,0.0,0.0,0.0,42.0,32.0,17.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-1,2013,1,5


In [65]:
df.describe()

,DAILYSunrise,DAILYSunset,wind_speed_average,precipitation,snowfall,snow_depth,temperature_max,temperature_min,fastest_2minute_windspeed,fog,...,ice_pellets,glaze_rime,smoke_haze,mist,drizzle,ice_fog,EVENT_TYPE,year,month,dayofyear
count,1794.000000,1794.000000,1794.000000,1794.000000,1794.000000,1794.000000,1794.000000,1794.000000,1794.000000,1794.000000,...,1794.000000,1794.000000,1794.000000,1794.000000,1794.000000,1794.000000,1794.000000,1794.000000,1794.000000,1794.000000
mean,561.280936,1786.552954,5.332656,0.126003,0.106916,0.593419,63.654404,49.250836,13.963229,0.283166,...,0.002787,0.003902,0.193423,0.015050,0.000557,0.001115,-0.898551,2014.964883,6.426421,180.132107
std,101.291318,105.020938,2.331693,0.361501,0.941535,2.402662,18.414538,17.139419,3.826866,0.450662,...,0.052734,0.062361,0.395091,0.121786,0.023610,0.033380,0.863053,1.401296,3.401780,103.991837
min,424.000000,1628.000000,0.670000,0.000000,0.000000,0.000000,15.000000,-1.000000,6.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,2013.000000,1.000000,1.000000
25%,452.000000,1706.000000,3.800000,0.000000,0.000000,0.000000,49.000000,36.000000,12.100000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,2014.000000,3.000000,90.000000
50%,545.000000,1807.000000,4.920000,0.000000,0.000000,0.000000,65.000000,51.000000,13.963229,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,2015.000000,6.000000,180.000000
75%,644.000000,1902.000000,6.490000,0.040000,0.000000,0.000000,80.000000,64.750000,16.100000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,2016.000000,9.000000,269.750000
max,720.000000,1931.000000,18.340000,4.970000,27.300000,22.000000,98.000000,83.000000,33.100000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,10.000000,2017.000000,12.000000,366.000000


In [66]:
df.dtypes

DATE                         datetime64[ns]
DAILYSunrise                          int64
DAILYSunset                           int64
wind_speed_average                  float64
precipitation                       float64
snowfall                            float64
snow_depth                          float64
temperature_max                     float64
temperature_min                     float64
fastest_2minute_windspeed           float64
fog                                 float64
heavy_fog                           float64
ice_pellets                         float64
glaze_rime                          float64
smoke_haze                          float64
mist                                float64
drizzle                             float64
ice_fog                             float64
EVENT_TYPE                            int64
year                                  int64
month                                 int64
dayofyear                             int64
dtype: object

## Data of one month

In [67]:
df = df[df['year'] == 2015]
df = df[df['month'] == 1]
df.head()

,DATE,DAILYSunrise,DAILYSunset,wind_speed_average,precipitation,snowfall,snow_depth,temperature_max,temperature_min,fastest_2minute_windspeed,...,ice_pellets,glaze_rime,smoke_haze,mist,drizzle,ice_fog,EVENT_TYPE,year,month,dayofyear
732,2015-01-01,720,1639,7.16,0.00,0.0,0.0,39.0,27.0,17.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-1,2015,1,1
733,2015-01-02,720,1640,7.16,0.00,0.0,0.0,42.0,35.0,15.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-1,2015,1,2
734,2015-01-03,720,1641,6.49,0.71,0.0,0.0,42.0,33.0,12.1,...,0.0,0.0,1.0,0.0,0.0,0.0,-1,2015,1,3
735,2015-01-04,720,1642,6.49,0.30,0.0,0.0,56.0,41.0,29.1,...,0.0,0.0,0.0,0.0,0.0,0.0,-1,2015,1,4
736,2015-01-05,720,1643,10.51,0.00,0.0,0.0,49.0,21.0,23.9,...,0.0,0.0,0.0,0.0,0.0,0.0,-1,2015,1,5


In [68]:
df.count()

DATE                         31
DAILYSunrise                 31
DAILYSunset                  31
wind_speed_average           31
precipitation                31
snowfall                     31
snow_depth                   31
temperature_max              31
temperature_min              31
fastest_2minute_windspeed    31
fog                          31
heavy_fog                    31
ice_pellets                  31
glaze_rime                   31
smoke_haze                   31
mist                         31
drizzle                      31
ice_fog                      31
EVENT_TYPE                   31
year                         31
month                        31
dayofyear                    31
dtype: int64

In [69]:
df.shape

(31, 22)

In [70]:
df.to_csv('clean_weather_data.csv')